In [16]:
from gwpy.time import to_gps, from_gps
from gwpy.segments import DataQualityFlag, SegmentList
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwpy.frequencyseries import FrequencySeries
import gwdatafind
import numpy as np
import pandas as pd
import os
import glob
import plotly.express as px
import plotly

In [17]:
def get_strain_data(starttime, endtime, duration, ifo='K1', source_gwf=None):
    """
    Retrieve strain data for the interferometer K1 over the given time interval.

    Parameters:
      - starttime (float): Start time (GPS seconds).
      - endtime (float): End time (GPS seconds).
      - duration (float): Duration (in seconds) of the interval.
      - ifo (str): Interferometer identifier (must be 'K1').
      - source_gwf (str or list, optional): Path(s) to the GWF file(s).

    Returns:
      - TimeSeries: The strain data for the given time range.
    """
    try:
        if ifo != 'K1':
            raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
        
        if source_gwf is None:
            # Use the provided starttime and duration.
            j = int(starttime)
            computed_endtime = j + duration  
            # Files are assumed to start every 32 seconds.
            first_file = int(j - (j % 32))
            last_file = int(computed_endtime - (computed_endtime % 32))
            # Compute a block directory (adjust if necessary)
            block = int((j - (j % 100000)) / 100000)
            # Build a list of expected strain data file names.
            
            strain_channel_files = [
                f"/data/KAGRA/raw/science/{block}/K-K1_R-{i}-32.gwf"
                for i in range(first_file, last_file + 1, 32)
            ]
            # Filter for files that exist.
            existing_files = [f for f in strain_channel_files if os.path.exists(f)]
            if len(existing_files) == 0:
                raise ValueError("No GWF files found for K1.")
            source_gwf = existing_files
            print(f"Selected strain data files: {source_gwf}")
        
        # Read the strain data using TimeSeries.read (a single channel)
        strain_channel = 'K1:CAL-CS_PROC_DARM_STRAIN_DBL_DQ'
        ht = TimeSeries.read(
            source=source_gwf,
            channel=strain_channel,
            start=starttime,
            end=endtime
        )
        return ht
    except Exception as e:
        print(f"An error occurred in get_strain_data: {e}")
        return None

In [ ]:
# strain_data = get_strain_data(1370183890, 1370183890+256, 256, 'K1', source_gwf=None)
# print(strain_data)

In [18]:
def get_frame_files(starttime, endtime, duration, ifo, directory=None):
    """
    Retrieve frame (witness channel) files for the interferometer K1 over the given time interval.

    Parameters:
      - starttime (float): Start time (GPS seconds).
      - endtime (float): End time (GPS seconds).
      - duration (float): Duration (in seconds) of the interval.
      - ifo (str): Interferometer identifier (must be 'K1').
      - directory (str, optional): Directory path for witness channel data.

    Returns:
      - list: A sorted list of file paths.
    """
    try:
        if ifo != 'K1':
            raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
        if directory is None:
            raise ValueError("A directory must be specified for 'K1' witness channel data.")
        if not os.path.isdir(directory):
            raise FileNotFoundError(f"Directory not found: {directory}")
        
        j = int(starttime)
        computed_endtime = j + duration
        first_file = int(j - (j % 32))
        last_file = int(computed_endtime - (computed_endtime % 32))
        block = int((j - (j % 100000)) / 100000)
        # Build a list of expected witness channel file names.
        witness_channel_files = [
            f"/data/KAGRA/raw/full/{block}/K-K1_C-{i}-32.gwf"
            for i in range(first_file, last_file + 1, 32)
        ]
        # Filter the list by checking file existence.
        files = sorted([f for f in witness_channel_files if os.path.exists(f)])
        return files
    except Exception as e:
        print(f"An error occurred in get_frame_files: {e}")
        return []

In [ ]:
# directory = '/data/KAGRA/raw/full/'
# frame_file = get_frame_files(1370183890, 1370183890+256, 256, 'K1', directory=directory)
# # print(list(check_witness_data))

In [ ]:
def calc_coherence(channel2, frame_file, start_time, end_time, fft, overlap, strain_data, channel1=None):
    t1 = to_gps(start_time)
    t2 = to_gps(end_time)

    if not isinstance(strain_data, TimeSeries):
        raise ValueError("The parameter `strain_data` must be a `TimeSeries` object.")
    try:
        ts2 = TimeSeriesDict.read(
            frame_file,
            channels=channel2,
            start=t1,
            end=t2
        )
    except FileNotFoundError:
        raise FileNotFoundError(f"Frame file not found: {frame_file}")
    except Exception as e:
        raise RuntimeError(f"Failed to read time series from {frame_file}: {e}")
    ts1 = strain_data.resample(ts2.sample_rate)
    coh = ts1.coherence(
        ts2,
        fftlength=fft,
        overlap=overlap,
        window='hann'
    )
    return coh

In [ ]:
def calc_coherence(channel2, frame_files, start_time, end_time, fft, overlap, strain_data, channel1=None):
    """
    Compute the coherence between strain data and witness (frame) data.

    Parameters:
      - channel2 (list or str): The witness channel(s) to be read.
      - frame_files (str or list): The file path(s) for the witness channel data.
      - start_time (float): Start time (GPS seconds).
      - end_time (float): End time (GPS seconds).
      - fft (float): The FFT length (in seconds) for the coherence calculation.
      - overlap (float): The overlap (in seconds) between FFT segments.
      - strain_data (TimeSeries): The strain data as a TimeSeries object.
      - channel1: (Optional) Not used in this implementation.

    Returns:
      - FrequencySeries: The coherence between the strain data and witness data.
    """
    # Convert start and end times to GPS-compatible times
    t1 = to_gps(start_time)
    t2 = to_gps(end_time)
    
    # Read the witness (frame) data using TimeSeriesDict.read.
    # Note: frame_files can be a single file or a list of files.
    try:
        ts2 = TimeSeriesDict.read(
            frame_files,
            channels=channel2,
            start=t1,
            end=t2
        )
    except FileNotFoundError:
        raise FileNotFoundError(f"Frame file not found: {frame_files}")
    except Exception as e:
        raise RuntimeError(f"Failed to read time series from {frame_files}: {e}")
    
    # Extract the sample rate from one of the witness channels.
    # (Assumes all witness channels share the same sample rate.)
    witness_sample_rate = list(ts2.values())[0].sample_rate

    # Resample the strain data if needed.
    if strain_data.sample_rate != witness_sample_rate:
        strain_data = strain_data.resample(witness_sample_rate)
    
    # Compute coherence using the resampled strain data and the witness data.
    coh = TimeSeries.coherence(
        strain_data,
        frame_files,
        fftlength=fft,
        overlap=overlap,
        window='hann'
        )
    return coh

In [43]:
from gwpy.time import to_gps, from_gps
from gwpy.segments import DataQualityFlag, SegmentList
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwpy.frequencyseries import FrequencySeries
import gwdatafind
import numpy as np
import pandas as pd
import os
import glob
import plotly.express as px
import plotly

def get_strain_data(starttime, endtime, duration, ifo='K1', source_gwf=None):
    """
    Retrieve strain data for the interferometer K1 over the given time interval.
    """
    try:
        if ifo != 'K1':
            raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
        
        if source_gwf is None:
            # Use the provided starttime and duration.
            j = int(starttime)
            computed_endtime = j + duration  
            # Files are assumed to start every 32 seconds.
            first_file = int(j - (j % 32))
            last_file = int(computed_endtime - (computed_endtime % 32))
            # Compute a block directory (adjust if necessary)
            block = int((j - (j % 100000)) / 100000)
            # Build a list of expected strain data file names.
            strain_channel_files = [
                f"/data/KAGRA/raw/science/{block}/K-K1_R-{i}-32.gwf"
                for i in range(first_file, last_file + 1, 32)
            ]
            # Filter for files that exist.
            existing_files = [f for f in strain_channel_files if os.path.exists(f)]
            if len(existing_files) == 0:
                raise ValueError("No GWF files found for K1.")
            source_gwf = existing_files
            print(f"Selected strain data files: {source_gwf}")
        
        # Read the strain data using TimeSeries.read (a single channel)
        strain_channel = 'K1:CAL-CS_PROC_DARM_STRAIN_DBL_DQ'
        ht = TimeSeries.read(
            source=source_gwf,
            channel=strain_channel,
            start=starttime,
            end=endtime
        )
        return ht
    except Exception as e:
        print(f"An error occurred in get_strain_data: {e}")
        return None

def get_frame_files(starttime, endtime, duration, ifo, directory=None):
    """
    Retrieve frame (witness channel) files for the interferometer K1 over the given time interval.
    """
    try:
        if ifo != 'K1':
            raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
        if directory is None:
            raise ValueError("A directory must be specified for 'K1' witness channel data.")
        if not os.path.isdir(directory):
            raise FileNotFoundError(f"Directory not found: {directory}")
        
        j = int(starttime)
        computed_endtime = j + duration
        first_file = int(j - (j % 32))
        last_file = int(computed_endtime - (computed_endtime % 32))
        block = int((j - (j % 100000)) / 100000)
        # Build a list of expected witness channel file names.
        witness_channel_files = [
            f"/data/KAGRA/raw/full/{block}/K-K1_C-{i}-32.gwf"
            for i in range(first_file, last_file + 1, 32)
        ]
        # Filter the list by checking file existence.
        files = sorted([f for f in witness_channel_files if os.path.exists(f)])
        return files
    except Exception as e:
        print(f"An error occurred in get_frame_files: {e}")
        return []

def calc_coherence(channel2, frame_files, start_time, end_time, fft, overlap, strain_data, channel1=None):
    """
    Compute the coherence between strain data and all witness (frame) channels.

    Parameters:
      - channel2 (list or str): The witness channel(s) to be read.
      - frame_files (str or list): The file path(s) for the witness channel data.
      - start_time (float): Start time (GPS seconds).
      - end_time (float): End time (GPS seconds).
      - fft (float): The FFT length (in seconds) for the coherence calculation.
      - overlap (float): The overlap (in seconds) between FFT segments.
      - strain_data (TimeSeries): The strain data as a TimeSeries object.
      - channel1: (Optional) Not used in this implementation.

    Returns:
      - dict: A dictionary where keys are witness channel names and values are the corresponding coherence FrequencySeries.
    """
    from gwpy.time import to_gps

    # Convert start and end times to GPS-compatible times.
    t1 = to_gps(start_time)
    t2 = to_gps(end_time)

    if not isinstance(strain_data, TimeSeries):
        raise ValueError("The parameter `strain_data` must be a TimeSeries object.")

    try:
        # Read the witness (frame) data into a TimeSeriesDict.
        ts2 = TimeSeriesDict.read(
            frame_files,
            channels=channel2,
            start=t1,
            end=t2
        )
    except FileNotFoundError:
        raise FileNotFoundError(f"Frame file(s) not found: {frame_files}")
    except Exception as e:
        raise RuntimeError(f"Failed to read time series from {frame_files}: {e}")

    coh = {}
    # Loop over each witness channel in the TimeSeriesDict.
    for chan_name, witness_ts in ts2.items():
        # Get the sample rate for the current witness channel.
        witness_sample_rate = witness_ts.sample_rate

        # Resample the strain data to match the witness channel's sample rate.
        ts1_resampled = strain_data.resample(witness_sample_rate)

        # Compute the coherence between the resampled strain data and the witness channel.
        coherence = ts1_resampled.coherence(
            witness_ts,
            fftlength=fft,
            overlap=overlap,
            window='hann'
        )
        coh[chan_name] = coherence

    return coh

## test ##
from gwpy.time import to_gps
import os

def run_coherence(channel_list, frame_files, starttime, endtime, strain_data, savedir, ifo, fft=10, overlap=5):
    """
    Calculate and save the coherence between strain data and witness channels.
    
    This function calculates the coherence between the given strain data and all witness channels 
    specified in `channel_list` (using the frame files). The coherence for each witness channel is 
    saved as a CSV file in a subdirectory of `savedir` based on the start time.
    
    Parameters:
      - channel_list (list): A list of witness channel names.
      - frame_files (str or list): The file path(s) for the witness channel data.
      - starttime (float): Start time in GPS seconds.
      - endtime (float): End time in GPS seconds.
      - strain_data (TimeSeries): The strain data as a TimeSeries object.
      - savedir (str): The base directory to save output files.
      - ifo (str): The interferometer identifier (must be 'K1').
      - fft (float, optional): The FFT length (in seconds) for the coherence calculation.
      - overlap (float, optional): The overlap (in seconds) between FFT segments.
    """
    # Convert start and end times to GPS times for naming and output.
    t1, t2 = to_gps(starttime), to_gps(endtime)
    
    # Create an output directory based on the start time.
    outdir = os.path.join(savedir, f'{t1}')
    if not os.path.exists(outdir):
        print(f"Creating the output directory: {outdir}")
        os.makedirs(outdir)
    
    # Determine the strain channel name.
    if ifo == 'K1':
        strain_channel = f'{ifo}:CAL-CS_PROC_DARM_STRAIN_DBL_DQ'
    else:
        raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
    
    try:
        print(f"Calculating coherence for witness channels: {', '.join(channel_list)} ...")
        # Call the updated calc_coherence with the full channel list.
        # Note: We pass the original starttime and endtime so that calc_coherence
        # can perform its own conversion.
        coherence_dict = calc_coherence(
            channel2=channel_list,
            frame_files=frame_files,
            start_time=starttime,
            end_time=endtime,
            fft=fft,
            overlap=overlap,
            strain_data=strain_data,
            channel1=None
        )
        
        # Iterate over each channel's coherence result and save the output.
        for chan, coh in coherence_dict.items():
            # Sanitize the channel name to create a valid filename.
            sanitized_channel = chan.replace(':', '_').replace('-', '_')
            output_file = os.path.join(outdir, f"{sanitized_channel}_{t1}_{t2}.csv")
            coh.write(output_file)
            print(f"Saved coherence data for {chan} to {output_file}")
            
    except Exception as e:
        print(f"Failed to calculate or save coherence: {e}")

## test ##

In [39]:
# ----- Usage Example -----
start_time = 1370183890
end_time = start_time + 256
channel_path = '/home/shu-wei.yeh/coherence-monitor/channel_files/K1/'
fft, overlap = 2, 1

# Read the channel CSV and convert the 'channel' column to a list.
channels_df = pd.read_csv(os.path.join(channel_path, 'volt_channels.csv'),
                          header=None, names=['channel'])
channel_list = channels_df['channel'].tolist()

duration = 256  # seconds

# Retrieve the strain data.
strain_data = get_strain_data(start_time, end_time, duration, ifo='K1', source_gwf=None)
if strain_data is None:
    raise RuntimeError("Strain data could not be loaded. Please check that the GWF files exist.")

# Retrieve the witness (frame) files.
witness_directory = '/data/KAGRA/raw/full/'  # Adjust this path as needed.
frame_files = get_frame_files(start_time, end_time, duration, ifo='K1', directory=witness_directory)
if not frame_files:
    raise RuntimeError("No witness frame files were found. Please check the directory and file availability.")

# Compute coherence using the full list of frame files.
coherence = calc_coherence(channel_list, frame_files, start_time, end_time, fft, overlap, strain_data)
# print("Coherence calculation successful!")
# print(coherence)

Selected strain data files: ['/data/KAGRA/raw/science/13701/K-K1_R-1370183872-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183904-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183936-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183968-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184000-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184032-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184064-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184096-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184128-32.gwf']


In [44]:
starttime = 1370183890
endtime = start_time + 256
savedir = './K1_automatic'
run_coh = run_coherence(channel_list, frame_files, starttime, endtime, strain_data, savedir, ifo='K1', fft=10, overlap=5)

Calculating coherence for witness channels: K1:PEM-VOLT_AS_TABLE_GND_OUT_DQ, K1:PEM-VOLT_IMCREFL_TABLE_GND_OUT_DQ, K1:PEM-VOLT_ISS_TABLE_GND_OUT_DQ, K1:PEM-VOLT_OMC_CHAMBER_GND_OUT_DQ, K1:PEM-VOLT_PSL_TABLE_GND_OUT_DQ, K1:PEM-VOLT_REFL_TABLE_GND_OUT_DQ ...
Saved coherence data for K1:PEM-VOLT_AS_TABLE_GND_OUT_DQ to ./K1_automatic/1370183890/K1_PEM_VOLT_AS_TABLE_GND_OUT_DQ_1370183890_1370184146.csv
Saved coherence data for K1:PEM-VOLT_IMCREFL_TABLE_GND_OUT_DQ to ./K1_automatic/1370183890/K1_PEM_VOLT_IMCREFL_TABLE_GND_OUT_DQ_1370183890_1370184146.csv
Saved coherence data for K1:PEM-VOLT_ISS_TABLE_GND_OUT_DQ to ./K1_automatic/1370183890/K1_PEM_VOLT_ISS_TABLE_GND_OUT_DQ_1370183890_1370184146.csv
Saved coherence data for K1:PEM-VOLT_OMC_CHAMBER_GND_OUT_DQ to ./K1_automatic/1370183890/K1_PEM_VOLT_OMC_CHAMBER_GND_OUT_DQ_1370183890_1370184146.csv
Saved coherence data for K1:PEM-VOLT_PSL_TABLE_GND_OUT_DQ to ./K1_automatic/1370183890/K1_PEM_VOLT_PSL_TABLE_GND_OUT_DQ_1370183890_1370184146.csv
Sav

In [45]:
def get_max_corr(output_dir, save=False):
    """
    Process all CSV files in `output_dir` to compute the maximum correlation value
    (and its corresponding frequency) over a default frequency band from 1 Hz to 200 Hz.
    
    Parameters:
      output_dir (str): Directory containing CSV files.
      save (bool): If True, return a DataFrame of channels with max correlation > 0.
    
    Returns:
      DataFrame: If save is True, a DataFrame with columns ['channel', 'max_correlation', 'frequency'];
                 otherwise, an empty DataFrame.
    """
    # Use an absolute path for consistency.
    output_dir = os.path.abspath(output_dir)
    files = glob.glob(os.path.join(output_dir, '*.csv'))
    if not files:
        raise FileNotFoundError(f"No CSV files found in directory: {output_dir}")
    
    vals = []
    for file_path in files:
        try:
            # Extract a channel name by splitting on 'DQ'
            base = os.path.basename(file_path)
            chan_name = base.split('DQ')[0] + 'DQ'
            
            # Read the FrequencySeries saved in the CSV file.
            fs = FrequencySeries.read(file_path)
            
            # Check that there are at least two frequency points to determine the spacing.
            if len(fs.frequencies) < 2:
                raise ValueError("Insufficient frequency points in FrequencySeries.")
            
            # Compute the frequency difference using the first two frequency points.
            n1, n2 = fs.frequencies.value[0], fs.frequencies.value[1]
            n_diff = n2 - n1
            
            # For the default frequency band of 1 Hz to 200 Hz:
            ind1, ind2 = int(1 / n_diff), int(200 / n_diff)
            fs_sub = fs[ind1:ind2]
            
            max_value = fs_sub.max().value
            max_value_frequency = fs_sub.frequencies[fs_sub.argmax()].value
            
            if save and max_value > 0:
                vals.append((chan_name, max_value, max_value_frequency))
        except Exception as e:
            print(f"Failed to process file {file_path}: {e}")
    
    if save:
        df_vals = pd.DataFrame(vals, columns=['channel', 'max_correlation', 'frequency'])
        return df_vals[df_vals['max_correlation'] > 0]
    
    return pd.DataFrame()


def get_max_corr_band(output_dir, flow=10, fhigh=20, save=False):
    """
    Process all CSV files in `output_dir` to compute the maximum correlation value
    (and its corresponding frequency) within a specified frequency band.
    
    Parameters:
      output_dir (str): Directory containing CSV files.
      flow (float): Lower bound of the frequency band.
      fhigh (float): Upper bound of the frequency band.
      save (bool): If True, return a DataFrame of channels with max correlation > 0.
    
    Returns:
      DataFrame: If save is True, a DataFrame with columns ['channel', 'max_correlation', 'frequency'];
                 otherwise, an empty DataFrame.
    """
    output_dir = os.path.abspath(output_dir)
    files = glob.glob(os.path.join(output_dir, '*.csv'))
    if not files:
        raise FileNotFoundError(f"No CSV files found in directory: {output_dir}")
    
    vals = []
    for file_path in files:
        try:
            base = os.path.basename(file_path)
            chan_name = base.split('DQ')[0] + 'DQ'
            
            fs = FrequencySeries.read(file_path)
            if len(fs.frequencies) < 2:
                raise ValueError("Insufficient frequency points in FrequencySeries.")
            
            n1, n2 = fs.frequencies.value[0], fs.frequencies.value[1]
            n_diff = n2 - n1
            
            # Compute indices corresponding to the desired frequency band [flow, fhigh]
            ind1, ind2 = int(flow / n_diff), int(fhigh / n_diff)
            fs_sub = fs[ind1:ind2]
            
            max_value = fs_sub.max().value
            max_value_frequency = fs_sub.frequencies[fs_sub.argmax()].value
            
            if save and max_value > 0:
                vals.append((chan_name, max_value, max_value_frequency))
        except Exception as e:
            print(f"Failed to process file {file_path}: {e}")
    
    if save:
        df_vals = pd.DataFrame(vals, columns=['channel', 'max_correlation', 'frequency'])
        return df_vals[df_vals['max_correlation'] > 0]
    
    return pd.DataFrame()

In [ ]:
def combine_csv(dir_path, ifo):
    """
    Combine CSV files in the given directory after filtering out unsafe channels.
    
    Parameters:
      - dir_path (str): Directory containing CSV files.
      - ifo (str): Interferometer identifier (used to filter unsafe channels).
    
    Returns:
      DataFrame: A combined DataFrame whose columns are renamed based on each file.
    """
    # Use absolute path for consistency.
    dir_path = os.path.abspath(dir_path)
    all_files = glob.glob(os.path.join(dir_path, "*.csv"))
    if not all_files:
        raise FileNotFoundError(f"No CSV files found in directory: {dir_path}")
    
    # Get the list of unsafe channels and sanitize their names.
    chan_removes = get_unsafe_channels(ifo=ifo)['channel']
    chan_removes = [chan.replace(':', '_').replace('-', '_') for chan in chan_removes]
    
    # Filter out files whose basenames start with any unsafe channel.
    filtered_files = [
        file for file in all_files
        if not any(os.path.basename(file).startswith(chan) for chan in chan_removes)
    ]
    if not filtered_files:
        raise ValueError("No valid CSV files found after filtering unsafe channels.")
    
    combined_data = []
    column_names = []
    for file_path in filtered_files:
        try:
            # Use part of the filename to generate column names.
            base_name = os.path.basename(file_path).split('_14')[0]
            column_freq = f"{base_name}_freq"
            column_corr = f"{base_name}_corr"
            df = pd.read_csv(file_path, header=None)
            combined_data.append(df)
            column_names.extend([column_freq, column_corr])
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
    
    if not combined_data:
        raise ValueError("No CSV files could be processed.")
    
    # Concatenate dataframes side by side.
    combined_df = pd.concat(combined_data, axis=1, ignore_index=True)
    if combined_df.shape[1] != len(column_names):
        raise ValueError("Mismatch between the number of columns and column names.")
    combined_df.columns = column_names
    return combined_df

def find_max_corr_channel(path, ifo, fft=10):
    """
    Find, for each frequency bin, the channels with the highest and second-highest coherence.
    
    Parameters:
      - path (str): Directory containing CSV files.
      - ifo (str): Interferometer identifier.
      - fft (float): FFT length used to compute frequency bins.
    
    Returns:
      DataFrame: A DataFrame with columns ['frequency', 'channel1', 'corr1', 'channel2', 'corr2'].
    """
    frame_df = combine_csv(path, ifo)
    if frame_df.empty:
        raise ValueError(f"No valid data found in the combined CSV files at {path}.")
    
    max_vals = []
    # Loop over each row (each frequency bin).
    for i in range(len(frame_df)):
        try:
            # Assume that every second column (starting with index 1) holds coherence values.
            corr_values = frame_df.iloc[i, 1::2]
            # Sort descending by coherence.
            corr_sorted = corr_values.sort_values(ascending=False)
            # Get the top two channel column indices.
            top_columns = corr_sorted.index[:2]
            # Reconstruct channel names from the column names.
            chan_names = [
                col.replace('_corr', '').replace(f'{ifo}_', f'{ifo}:')
                for col in top_columns
            ]
            max_corr_vals = corr_sorted.iloc[:2].tolist()
            # Compute the frequency (row index divided by the FFT length).
            freq = i / fft
            max_vals.append((freq, chan_names[0], max_corr_vals[0],
                             chan_names[1], max_corr_vals[1]))
        except Exception as e:
            print(f"Error processing row {i}: {e}")
            continue
    df_max = pd.DataFrame(max_vals, columns=['frequency', 'channel1', 'corr1', 'channel2', 'corr2'])
    return df_max

def plot_max_corr_chan(path, fft, ifo, flow=0, fhigh=200, duration=256):
    """
    Plot the highest and second-highest coherence channels across frequency bins.
    
    Parameters:
      - path (str): Directory containing CSV files.
      - fft (float): FFT length (used in frequency calculation).
      - ifo (str): Interferometer identifier.
      - flow (float): Lower frequency bound for plotting.
      - fhigh (float): Upper frequency bound for plotting.
      - duration (int): Duration used in plot titles.
    
    Returns:
      DataFrame: The DataFrame of maximum correlation channel data.
    """
    try:
        # Assume the directory name is a timestamp.
        time_ = int(os.path.basename(os.path.normpath(path)))
        vals = find_max_corr_channel(path=path, fft=fft, ifo=ifo)
        print("Data acquired; generating plots...")
        # Filter rows by frequency.
        vals = vals[(vals['frequency'] >= flow) & (vals['frequency'] <= fhigh)]
        
        # Apply a grouping function to channels (assumes give_group_v2 is defined).
        vals['group1'] = vals['channel1'].apply(give_group_v2)
        vals['group2'] = vals['channel2'].apply(give_group_v2)
        
        # Plot highest coherence channel.
        fig1 = px.scatter(
            vals,
            x="frequency",
            y="corr1",
            hover_data=['channel1'],
            color="group1",
            labels={"corr1": "Max Coherence", "frequency": "Frequency [Hz]"}
        )
        fig1.update_layout(
            title={
                "text": f"Highest Coherence Channel at Each Frequency ({time_} to {time_ + duration})",
                "font": {"family": "Courier New, monospace", "size": 28, "color": "RebeccaPurple"}
            },
            font_size=28
        )
        
        # Plot second-highest coherence channel.
        fig2 = px.scatter(
            vals,
            x="frequency",
            y="corr2",
            hover_data=['channel2'],
            color="group2",
            labels={"corr2": "Second Max Coherence", "frequency": "Frequency [Hz]"}
        )
        fig2.update_layout(
            title={
                "text": f"Second Highest Coherence Channel at Each Frequency ({time_} to {time_ + duration})",
                "font": {"family": "Courier New, monospace", "size": 28, "color": "RebeccaPurple"}
            }
        )
        
        # Create a subdirectory for plots.
        plot_dir = os.path.join(path, "plots")
        os.makedirs(plot_dir, exist_ok=True)
        plotly.offline.plot(fig1, filename=os.path.join(plot_dir, f'channels_coh_{time_}_a.html'))
        plotly.offline.plot(fig2, filename=os.path.join(plot_dir, f'channels_coh_{time_}_b.html'))
        print("Plots saved successfully.")
        return vals
    except Exception as e:
        print(f"An error occurred during plotting: {e}")
        return pd.DataFrame()

In [20]:
# Import your functions (adjust the module name if needed)
# from your_module import get_strain_data, get_frame_files, calc_coherence

# --- Provided parameters ---
start_time = 1370183890
end_time = start_time + 256
channel_path = '/home/shu-wei.yeh/coherence-monitor/channel_files/K1/'
fft, overlap = 2, 1

In [21]:
# Read the channel CSV and convert the channel column to a list of strings.
# (TimeSeriesDict.read expects channel names as a list of strings.)
channels_df = pd.read_csv(os.path.join(channel_path, 'volt_channels.csv'),
                          header=None, names=['channel'])
channel_list = channels_df['channel'].tolist()
# print(channel_list)

In [26]:
# --- Retrieve the strain data ---
# You may need to set source_gwf to None (or to a list of valid files)
duration = 256  # seconds
strain_data = get_strain_data(start_time, end_time, duration, ifo='K1', source_gwf=None)
# print(len(strain_data))

if strain_data is None:
    raise RuntimeError("Strain data could not be loaded. Check that the GWF files exist.")

Selected strain data files: ['/data/KAGRA/raw/science/13701/K-K1_R-1370183872-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183904-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183936-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370183968-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184000-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184032-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184064-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184096-32.gwf', '/data/KAGRA/raw/science/13701/K-K1_R-1370184128-32.gwf']


In [25]:
# --- Retrieve witness (frame) files ---
# Make sure to update witness_directory to a valid directory on your system.
witness_directory = '/data/KAGRA/raw/full/'  # Adjust this path as needed.
frame_files = get_frame_files(start_time, end_time, duration, ifo='K1', directory=witness_directory)

if not frame_files:
    raise RuntimeError("No witness frame files were found. Please check the directory and file availability.")

# For this test, we take the first available frame file.
frame_file = frame_files
print(len(frame_file))

9


In [27]:
# --- Calculate Coherence ---
try:
    # Note: We pass 'channel_list' instead of the raw DataFrame.
    coherence = calc_coherence(channel_list, frame_file, start_time, end_time, fft, overlap, strain_data)
    print("Coherence calculation successful!")
    print(coherence)
except Exception as e:
    print(f"An error occurred during coherence calculation: {e}")

An error occurred during coherence calculation: 'TimeSeriesDict' object has no attribute 'sample_rate'


In [ ]:
def run_coherence(channel_list, frame_files, starttime, endtime, strain_data, savedir, ifo, fft=fft, overlap=overlap):
    t1, t2 = to_gps(starttime), to_gps(endtime)
    savedir = os.path.join(savedir, f'{t1}')
    if not os.path.exists(savedir):
        print(f"Creating the output directory: {savedir}")
        os.makedirs(savedir)
    if ifo == 'K1':
        strain_channel = f'{ifo}:CAL-CS_PROC_DARM_STRAIN_DBL_DQ'
    else:
        raise ValueError(f"Unsupported interferometer: {ifo}. Must be 'K1'.")
    for channel in channel_list:
        try:
            print(f"Calculating coherence between {strain_channel} and {channel}...")
            coh = calc_coherence(
                strain_data=strain_data,
                channel1=None,
                channel2=channel,
                frame_file=frame_files,
                start_time=t1,
                end_time=t2,
                fft=fft,
                overlap=overlap
            )
            sanitized_channel = channel.replace(':', '_').replace('-', '_')
            output_file = os.path.join(savedir, f"{sanitized_channel}_{t1}_{t2}.csv")
            coh.write(output_file)
            print(f"Saved coherence data to {output_file}")
        except Exception as e:
            print(f"Failed to calculate or save coherence for {channel}: {e}")

In [ ]:
start_time = 1370183890
end_time = start_time + 256
channel_path = '/home/shu-wei.yeh/coherence-monitor/channel_files/K1/'
channel2 = pd.read_csv(os.path.join(channel_path, 'volt_channels.csv'), header=None, names=['channel'])
fft, overlap = 2, 1

In [ ]:
check_calc_coherence = calc_coherence(channel2, frame_file, start_time, end_time, fft, overlap, strain_data, channel1=None)

In [ ]:

# ht_data = get_strain_data(time_, time_ + duration, duration, ifo=ifo)
# print("Got h(t) data")

# def give_group(a):
#     return a.split('_')[1]

# def get_coherence_volt(channel_list, ifo, t0, strain_data, savedir, duration):
#     # Get witness files from the specified directory over the interval [t0, t0+duration)
#     files_ = get_frame_files(t0, t0 + duration, duration, ifo=ifo, directory='/data/KAGRA/raw/full/')
#     print("Got {} files".format(len(files_)))
#     if not files_:
#         raise ValueError("No frame files found.")
#     # *** FIX: Pass the entire list of files, not just the first file ***
#     run_coherence(
#         channel_list=channel_list,
#         frame_files=files_[0],
#         starttime=t0,
#         endtime=t0 + duration,
#         ifo=ifo,
#         strain_data=strain_data,
#         savedir=savedir
#     )
#     return

In [ ]:
channel = 'K1:PEM-VOLT_REFL_TABLE_GND_OUT_DQ'
group = channel.split(':')[1].split('_')[0]
print(group)

PEM-VOLT
